In [ ]:
# First, create a Python list
wordList = ["doug", "jon", "sameer", "eliano", "richard"]

In [ ]:
# Now, we'll create an RDD from the Python list with 4 partitions
wordsRDD = sc.parallelize(wordList, 4)

In [ ]:
# Why won't this print anything?
def println(value):
   print(value)
    
wordsRDD.map(println)

In [ ]:
# This will print something... but where?
wordsRDD.map(println).collect()
# Let's look in (executor) logs

In [ ]:
# Now, we'll create an RDD from the array, requesting 6 partitions
wordsRDD = sc.parallelize(wordList, 6)

In [ ]:
wordsRDD.collect()

In [ ]:
sc

In [ ]:
sc.version

In [ ]:
sc.appName

In [ ]:
sc.defaultParallelism

In [ ]:
datapath = "/data/training/tom_sawyer.txt" # https://www.gutenberg.org/ebooks/74

In [ ]:
sc.textFile(datapath, 2).collect()

In [ ]:
# Note, collect does not return an RDD (it returns an Array)
ebook = sc.textFile(datapath, 2).collect()

In [ ]:
# Why doesnt this work?
ebook.count()

In [ ]:
print('type of ebook: {0}'.format(type(ebook)))

In [ ]:
# Why doesnt this show up in the UI
ebookRDD = sc.textFile(datapath, 2)

In [ ]:
ebookRDD.count()

In [ ]:
ebookRDD.getNumPartitions()

In [ ]:
print('type of ebookRDD: {0}'.format(type(ebookRDD)))

In [ ]:
ebookRDD

In [ ]:
# Let's use RDD.glom to look at the distribution of data in our partitions

# What does glom do? Replaces each partition in the RDD with a single container that contains all of the partition's previous items
# http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.glom

glomDemo = sc.parallelize([1,2,3,4,5,6,7,8,9], 3)
glomDemo.glom().collect()

# How does it work? This is a good one to guess about, and then check the source

In [ ]:
ebookRDD.glom().collect()

In [ ]:
# Count the number of items in each partition.
for (p, i) in ebookRDD.glom().zipWithIndex().collect():
  print('%d: %d items(s)' % (i, len(p)))

In [ ]:
sc.textFile(datapath, 2).flatMap(lambda x: x.split(' ')).collect()

In [ ]:
sc.textFile(datapath, 2).flatMap(lambda x: x.split(' ')).map(lambda s: (s, 1)).collect()

In [ ]:
sc.textFile(datapath, 2).flatMap(lambda x: x.split(' ')).map(lambda s: (s, 1)).reduceByKey(lambda x, y: x + y).collect()

In [ ]:
# why doesnt this show up?
cachedBookRDD = sc.textFile(datapath, 2).cache()

In [ ]:
cachedBookRDD.setName("My Cached Book") 

In [ ]:
cachedBookRDD.count()

In [ ]:
cachedBookRDD.unpersist()

In [ ]:
from pyspark.storagelevel import StorageLevel
cachedBookSerializedRDD = sc.textFile(datapath, 2).persist(StorageLevel.MEMORY_ONLY_SER) #same thing as above, since we're using Python

In [ ]:
cachedBookSerializedRDD.setName("My Serialized Cached Book")

In [ ]:
cachedBookSerializedRDD.count()

In [ ]:
sc.textFile(datapath, 2).flatMap(lambda x: x.split(' ')).map(lambda s: (s, 1)).reduceByKey(lambda x, y: x + y).collect()

In [ ]:
tempRDD = sc.textFile(datapath, 2).repartition(4)

In [ ]:
tempRDD.getNumPartitions()

In [ ]:
# It simply counts the number of items in each partition.
for (p, i) in tempRDD.glom().zipWithIndex().collect():
  print('%d: %d items(s)' % (i, len(p)))

In [ ]:
sc.textFile(datapath, 2).repartition(12).cache().flatMap(lambda x: x.split(' ')).map(lambda s: (s, 1)).reduceByKey(lambda x, y: x + y).collect()

In [ ]:
sc.textFile(datapath, 2).repartition(4).cache().flatMap(lambda x: x.split(' ')).map(lambda s: (s, 1)).reduceByKey(lambda x, y: x + y).collect()

In [ ]:
myRDD = sc.textFile(datapath, 2).repartition(12).cache().flatMap(lambda x: x.split(' ')).map(lambda s: (s, 1)).reduceByKey(lambda x, y: x + y, numPartitions=8)
myRDD.collect()

In [ ]:
myRDD.getNumPartitions()

In [ ]:
myRDD.count() 
# this job uses data that is downstream from a shuffle we recently did -- Spark should use those shuffle output files to take a shortcut
# let's look in the UI for skipped stages